In [2]:
import pandas as pd
import numpy as np
import random as rd
from math import ceil,floor,factorial
from scipy.optimize import minimize

In [4]:
#Cria um dataframe com todos os times que participaram de algum campeonato mundial ou campeonatos regionais de 2018 a 2020
Times = pd.read_excel('TIMES.xlsx')
Times.set_index('Número', inplace = True)
Times.fillna('NA', inplace = True)
Times['Porcentagem de Vitorias'] = pd.Series(np.random.randn(len(list(Times.Sigla))), index=Times.index)

#Inicializando um dicionario global que ira conter as proficiencias de todos os times
proficiencia = {}

#Inicializando um conjunto global que contem os campeonatos que tem sua fase de grupos em formato MD3
md3_cships = ('LPL', 'LCK', 'VCS')

In [3]:
def liga_usada(liga,times,sigla):
    #Prepara os parametros para a computacao da verossimilhanca
    global x0,BOUND,bnds
    x0 = list(np.random.uniform(0,1
                                ,len(times)))
    BOUND = (0.001,5.0)
    bnds = tuple([BOUND]*len(times))
    return (liga,times,sigla)

In [4]:
def vero(x, liga, times, sigla, gtbc = 0):
    #Computa a verossimilhanca e calcula os estimadores de maxima verossimilhanca para as proficiencias dos times
    y = 0
    proficiencia[sigla] = {}
    for time in times:
        proficiencia[sigla][time] = x[y]
        y+=1
    multiplicatorio = 1
    #Defino a constante de esquecimento k, obtida experimentalmente
    k = 0.075
    #Defino quantos jogos serao considerados no calculo da verossimilhanca
    games = gtbc if gtbc else liga.shape[0]
    for game in range (0,games):
        w = liga.iloc[game].Week
        pv = proficiencia[sigla][liga.iloc[game].Winner]
        pl = proficiencia[sigla][liga.iloc[game].Loser]
        
        #Diferenciando play-offs de fase regular
        if w>0:
            cte = 1 - (max(list(liga.Week)) - w)*k
        else:
            cte = 1+k
        multiplicatorio *= (pv/(pv+pl))**cte
    
    return -np.log(multiplicatorio)


def restricao1(x):
    return x[0]-1        

def restricao2(x):
    return sum(x)-5

def restricao3(x):
    return sum(x)-50

con1 = {'type':'eq','fun':restricao1}
con2 = {'type':'eq','fun':restricao2}
con3 = {'type':'eq','fun':restricao3}
cons = [con2]

In [5]:
def probabilidade(A,B,n,provisorio = False):
    #Calcula as probabilidades do confronto entre A e B
    A_list = []
    B_list = []
    i,k = 0,floor(n/2)
    if provisorio:
        x = salvar[A]/(salvar[A]+salvar[B])
    else:
        siglaA = list(Times[Times.Sigla==A]['Região'])[0]
        siglaB = list(Times[Times.Sigla==B]['Região'])[0]
        x = proficiencia[siglaA][A]/(proficiencia[siglaA][A]+proficiencia[siglaB][B])
    print("         {} vs {}".format(A,B))
    while (ceil(n/2)+i) <= n:
        if i == 0:
            A_list.append(x**(ceil(n/2))*(1-x)**(i))
            B_list.append(x**(i)*(1-x)**(ceil(n/2)))
        else:
            p = factorial(k)/(factorial(i)*factorial(k-i)) 
            A_list.append(p*x**(ceil(n/2))*(1-x)**(i))
            B_list.append(p*x**(i)*(1-x)**(ceil(n/2)))
        i+=1
        k+=1
    if n==1:
        print('{} tem {:.2f} % de chance de vitória'.format(A,100*sum(A_list)))
        print('Enquanto {} tem {:.2f} %.'.format(B,100*sum(B_list)))
    else:    
        print('{} tem {:.2f} % de chance de vitória.Sendo:'.format(A,100*sum(A_list)))
        for b in range(0,ceil(n/2)):
            print('{:.2f} % a de ser {}-{}'.format(100*A_list[b],ceil(n/2),b))
        print('Enquanto {} tem {:.2f} %. Sendo:'.format(B,100*sum(B_list)))
        for b in range(0,ceil(n/2)):
            print('{:.2f} % a de ser {}-{}'.format(100*B_list[b],ceil(n/2),b))
    return sum(A_list)

In [6]:
def estimar_proficiencias(regiao, championship_dic, teams_dict, initials_dict, gtbc = 0):
    #Estima as proficiencias dos times da regiao inserida
    liga,times,sigla = liga_usada(championship_dic[regiao], teams_dict[regiao], initials_dict[regiao])
    sol = minimize(vero, x0, args=(liga, times, sigla, gtbc), method='SLSQP', constraints=cons , bounds=bnds)

In [ ]:
def calcular_acertos_em_determinada_semana(regiao, championship_dic, teams_dict, initials_dict, semana = 0, play_off_games_to_be_ignored = 0):
        if semana:
            games_to_be_considered = championship_dic[regiao][(championship_dic[regiao].Week >= 1) & (championship_dic[regiao].Week < semana)].shape[0]
            jogos_da_semana = championship_dic[regiao][championship_dic[regiao].Week == semana]
        else:
            games_to_be_considered = championship_dic[regiao].shape[0] - play_off_games_to_be_ignored
            jogos_da_semana = championship_dic[regiao].tail(play_off_games_to_be_ignored)
        
        estimar_proficiencias(regiao, championship_dic, teams_dict, initials_dict, gtbc = games_to_be_considered)
        acertos = 0
        erros = 0
        
        if semana:
            for game in range (0, jogos_da_semana.shape[0]):
                w,l = jogos_da_semana.iloc[game].Winner, jogos_da_semana.iloc[game].Loser
                pv, pl = proficiencia[initials_dict[regiao]][w], proficiencia[initials_dict[regiao]][l]     
                guess = probabilidade(w, l, 3) if regiao in md3_cships else pv/(pv+pl)                
                
                if guess > 0.45:
                    acertos += 1
                else:
                    erros += 1

            print("Na semana {} do campeonato {}, modelo acertou {} de {} jogos, resultando em {:.2f}% de acerto.".format(semana, regiao, acertos, acertos+erros, 100*acertos/(acertos+erros)))
            return acertos, erros
        else: 
            print("Favor calcular manualmente o resultado.")